In [ ]:
import torch
from fourier_scaffold import FourierScaffold
from graph_utils import fourier_plot_probabilities_complex

In [ ]:
shapes = [[5, 5], [7, 7]]
device = "cuda"
scaffold = FourierScaffold(shapes=torch.tensor(shapes), D=100000, device=device)

In [ ]:
scaffold.shift(v=torch.tensor([30, 30]))

In [ ]:
scaffold.smooth()

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(subplot_kw={"projection": "polar"})
ax.set_ylim(0,1)
fourier_plot_probabilities_complex(scaffold, ax)
plt.show()